In [20]:
import os
import fitz
from tqdm import tqdm

In [21]:
def check_rate_function(rate_1, rate_2, possible_rate):
        # 檢查兩種方法提取的評價是否相同
    if rate_1 == rate_2 and rate_1!= None:
        return rate_1
    else:
        if rate_1 in possible_rate:
            return rate_1
        elif rate_2 in possible_rate:
            return rate_2
        else :
            return 'NULL'
            

In [22]:
zip_path = r'..\file\台新 新聞評析'
count_true = 0
possible_rate = ['長期持有', '中立', '買進', '強力買進', '買 進']
rate_1 = None
rate_2 = None
rate = None
for directory in tqdm(os.listdir(zip_path)):
    directory_path = os.path.join(zip_path, directory)
    try:
        # 使用with語句讀取PDF文件，確保在使用後自動關閉文件。
        with fitz.open(directory_path) as doc:
            mat = fitz.Matrix(1, 1)
            page = doc.load_page(0)  # 加載文檔的第0頁
            rect = page.rect
            try:
                # 檢查是否由中信投顧出版
                page_check_source = doc.load_page(-1)
                text_check_source = page_check_source.get_text()
                if '台新證券投資顧問' in text_check_source :
                    try:
                        # 裁剪並提取文本的第一種方法
                        clip_new_version_1 = fitz.Rect(0, 0, 230, 180)
                        text_new_version_1 = page.get_text(clip=clip_new_version_1).strip()
                        text_new_version_1 = text_new_version_1.split('投資評等')[1].strip()
                        rate_1 = text_new_version_1.split('\n')[0].strip()
                    # 如果出現任何錯誤，設置rate_1為None。
                    except:
                        rate_1 = None
                    try:
                        # 裁剪文本的第二種方法
                        clip_new_version_2 = fitz.Rect(110, 85, 220, 120)
                        text_new_version_1 = page.get_text(clip=clip_new_version_2).strip()
                        rate_2 = text_new_version_1.split('\n')[0].strip()
                    # 如果出現任何錯誤，設置rate_2為None。
                    except:
                        rate_2 = None
                elif doc.page_count == 1:
                    page_check_source_news = doc.load_page(0)
                    clip_check_source_news = fitz.Rect(0, 500, rect.width, rect.height)
                    text_check_source_news = page_check_source_news.get_text(clip=clip_check_source_news)
                    if '台新投資' in text_check_source_news :
                        # 報告為新聞評析
                        try:
                            clip_news_version_1 = fitz.Rect(0, 0, 240, rect.height)
                            pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_news_version_1)
                            pix.save("page-clip_news_version_1.png")
                            text_news_version_1 = page.get_text(clip=clip_news_version_1, sort=True).strip()
                            text_news_version_1 = text_news_version_1.split('投資評等')[1].strip()
                            rate_1 = text_news_version_1.split('\n')[0].strip()
                        except:
                            rate_1 = None
                        # 報告為新聞評析
                        try:
                            clip_news_version_2 = fitz.Rect(140, 145, 240, 190)
                            pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_news_version_2)
                            pix.save("page-clip_news_version_2.png")
                            rate_2 = page.get_text(clip=clip_news_version_2).strip()
                        except:
                            rate_2 = None
                else:
                    # False
                    print(f'{directory_path} is not posted by 台新投顧')
                # if rate_1 == rate_2 and rate_1 not in possible_rate:
                #     possible_rate.append(rate_1)
                rate = check_rate_function(rate_1, rate_2, possible_rate)
                if rate == 'NULL' :
                    print(directory_path, rate,
                        f'rate_1={rate_1}, rate_2={rate_2}')
                elif rate != None :
                    count_true += 1
                    # print(directory_path, rate,
                    #         f'rate_1={rate_1}, rate_2={rate_2}')
                else :
                    print(directory_path, rate,
                            f'rate_1={rate_1}, rate_2={rate_2}')
                if rate not in possible_rate :
                    # count_true += 1
                    print(directory_path, rate,
                        f'rate_1={rate_1}, rate_2={rate_2}')
            except:
                print(directory_path, 'fail')
    except:
        print(directory_path, 'fail')

# 計算成功和失敗的數量並打印
count_false = len(os.listdir(zip_path)) - count_true
success_rate = round((count_true/len(os.listdir(zip_path))), 3)*100
print(f'共{len(os.listdir(zip_path))}件, 成功{count_true}件, 失敗{count_false}件, 成功率:{success_rate}%')

 14%|█▎        | 33/244 [00:03<00:21, 10.00it/s]

..\file\台新 新聞評析\1590_亞德客-KY_2018-12-06_台新投顧_台新投顧20181206新聞評析-亞德客(1590)-亞德客11月業績回神.pdf 325 rate_1=325, rate_2=325


100%|██████████| 244/244 [00:25<00:00,  9.43it/s]

共244件, 成功244件, 失敗0件, 成功率:100.0%


In [23]:
# # check header
# zip_path = '台新'
# count_true = 0

# for directory in os.listdir(zip_path):
#     directory_path = os.path.join(zip_path, directory)
#     try:
#         # 使用with語句讀取PDF文件，確保在使用後自動關閉文件。
#         with fitz.open(directory_path) as doc:
#             if doc.page_count > 1 :
#                 mat = fitz.Matrix(1, 1)
#                 page = doc.load_page(-1)  # 加載文檔的第-1頁
#                 rect = page.rect
#                 try:
#                     clip = fitz.Rect(0, 0, rect.width, rect.height)
#                     text = page.get_text(clip=clip)
#                     if '台新證券投資顧問' in text :
#                         count_true+=1
#                         # print(directory_path, 'True')
#                     else:
#                         print(directory_path, 'False')
#                 except:
#                     pass
#             else:
#                 print(directory_path, 'wrong')
#     except:
#         print(directory_path, 'fail')

# # 計算成功和失敗的數量並打印
# count_false = len(os.listdir(zip_path)) - count_true
# print(f'共{len(os.listdir(zip_path))}件, 成功{count_true}件, 失敗{count_false}件')

In [37]:
import fitz
doc = fitz.open(r'..\file\台新 新聞評析\1590_亞德客-KY_2018-12-06_台新投顧_台新投顧20181206新聞評析-亞德客(1590)-亞德客11月業績回神.pdf')
mat = fitz.Matrix(1, 1)
page = doc.load_page(0)  # loads page number 'pno' of the document (0-based)
rect = page.rect
pix = page.get_pixmap(matrix=mat, alpha=False)
pix.save("page-%i.png" % page.number)

In [38]:
page_check_source = doc.load_page(-1)
clip_check_source = fitz.Rect(0, 0, rect.width, rect.height)
pix = page_check_source.get_pixmap(matrix=mat, alpha=False, clip=clip_check_source)
pix.save("page-clip_check_source.png")
text_check_source = page_check_source.get_text(clip=clip_check_source)
if '台新證券投資顧問' in text_check_source :
    print('True')

In [39]:
# 檢查是否為個股報告
if '個股報告' in text_check_source :
    print('True')

In [40]:
# 檢查報告版本
clip_check_version= fitz.Rect(370, 80, 450, 200)
pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_check_version)
pix.save("page-clip_check_version.png")
text_check_version = page.get_text(clip=clip_check_version).strip()
if '投資評等' in text_check_version:
   print('報告為舊版')

In [41]:
# 報告為舊版
try:
    text_old_version_1 = text_check_version
    text_old_version_1 = text_old_version_1.split('投資評等')[1].strip()
    rate_1 = text_old_version_1.split('\n')[0].strip()
except:
    rate_1 = None
rate_1

In [42]:
# 報告為舊版
try:
    clip_old_version_2 = fitz.Rect(370, 120, 430, 150)
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_version_2)
    pix.save("page-clip_old_version_2.png")
    rate_2 = page.get_text(clip=clip_old_version_2).strip()
except:
    rate_2 = None
rate_2

''

In [43]:
# 報告為新版
try:
    # 裁剪並提取文本的第一種方法
    clip_new_version_1 = fitz.Rect(0, 0, 230, 180)
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_new_version_1)
    pix.save("page-clip_new_version_1.png")
    text_new_version_1 = page.get_text(clip=clip_new_version_1).strip()
    text_new_version_1 = text_new_version_1.split('投資評等')[1].strip()
    rate_1 = text_new_version_1.split('\n')[0].strip()
# 如果出現任何錯誤，設置rate_1和rate_2為None。
except:
    rate_1 = None
rate_1

'325'

In [44]:
# 報告為新版
try:
    # 裁剪文本的第二種方法
    clip_new_version_2 = fitz.Rect(110, 85, 220, 120)
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_new_version_2)
    pix.save("page-clip_new_version_2.png")
    text_new_version_1 = page.get_text(clip=clip_new_version_2).strip()
    rate_2 = text_new_version_1.split('\n')[0].strip()
# 如果出現任何錯誤，設置rate_2為None。
except:
    rate_2 = None
rate_2

''

In [49]:
import fitz
doc = fitz.open(r'..\file\台新 新聞評析\1590_亞德客-KY_2018-12-06_台新投顧_台新投顧20181206新聞評析-亞德客(1590)-亞德客11月業績回神.pdf')
mat = fitz.Matrix(1, 1)
page = doc.load_page(0)  # loads page number 'pno' of the document (0-based)
rect = page.rect
pix = page.get_pixmap(matrix=mat, alpha=False)
pix.save("page-%i.png" % page.number)

In [50]:
if doc.page_count == 1:
    print('新聞評析')

新聞評析


In [51]:
page_check_source_news = doc.load_page(0)
clip_check_source_news = fitz.Rect(0, 500, rect.width, rect.height)
pix = page_check_source_news.get_pixmap(matrix=mat, alpha=False, clip=clip_check_source_news)
pix.save("page-clip_check_source_news.png")
text_check_source_news = page_check_source_news.get_text(clip=clip_check_source_news)
if '台新投資' in text_check_source_news :
    print('True')
text_check_source_news

True


'誠信  承諾  創新  合作 \n【歡迎加入Line@  台新投資 雪亮觀點】 \n'

In [52]:
# 報告為新聞評析
try:
    clip_news_version_1 = fitz.Rect(0, 0, 240, rect.height)
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_news_version_1)
    pix.save("page-clip_news_version_1.png")
    text_news_version_1 = page.get_text(clip=clip_news_version_1, sort=True).strip()
    text_news_version_1 = text_news_version_1.split('投資評等')[1].strip()
    rate_1 = text_news_version_1.split('\n')[0].strip()
except:
    rate_1 = None
rate_1


'325'

In [53]:
# 報告為新聞評析
try:
    clip_news_version_2 = fitz.Rect(140, 145, 240, 190)
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_news_version_2)
    pix.save("page-clip_news_version_2.png")
    rate_2 = page.get_text(clip=clip_news_version_2).strip()
except:
    rate_2 = None
rate_2

'325'